In [5]:
import nltk

In [6]:
raw = "Her Santa Fe Opera debut in 2005 was as Nuria in the revised edition of Golijov's Ainadamar. She sang on the subsequent Deutsche Grammophon recording of the opera. For his opera Doctor Atomic, Adams rewrote the role of Kitty Oppenheimer, originally a mezzo-soprano role, for soprano voice, and Rivera sang the rewritten part of Kitty Oppenheimer at Lyric Opera of Chicago, De Nederlandse Opera, and the Metropolitan Opera., all in 2007. She has since sung several parts and roles in John Adams' works, including the soprano part in El Ni*o, and the role of Kumudha in A Flowering Tree in the Peter Sellars production at the New Crowned Hope Festival in Vienna."

In [30]:
sents = [nltk.word_tokenize(sent) for sent in nltk.sent_tokenize(raw)]
tagged_sents = [nltk.pos_tag(sent) for sent in sents]
print(sents)

[['Her', 'Santa', 'Fe', 'Opera', 'debut', 'in', '2005', 'was', 'as', 'Nuria', 'in', 'the', 'revised', 'edition', 'of', 'Golijov', "'s", 'Ainadamar', '.'], ['She', 'sang', 'on', 'the', 'subsequent', 'Deutsche', 'Grammophon', 'recording', 'of', 'the', 'opera', '.'], ['For', 'his', 'opera', 'Doctor', 'Atomic', ',', 'Adams', 'rewrote', 'the', 'role', 'of', 'Kitty', 'Oppenheimer', ',', 'originally', 'a', 'mezzo-soprano', 'role', ',', 'for', 'soprano', 'voice', ',', 'and', 'Rivera', 'sang', 'the', 'rewritten', 'part', 'of', 'Kitty', 'Oppenheimer', 'at', 'Lyric', 'Opera', 'of', 'Chicago', ',', 'De', 'Nederlandse', 'Opera', ',', 'and', 'the', 'Metropolitan', 'Opera.', ',', 'all', 'in', '2007', '.'], ['She', 'has', 'since', 'sung', 'several', 'parts', 'and', 'roles', 'in', 'John', 'Adams', "'", 'works', ',', 'including', 'the', 'soprano', 'part', 'in', 'El', 'Ni', '*', 'o', ',', 'and', 'the', 'role', 'of', 'Kumudha', 'in', 'A', 'Flowering', 'Tree', 'in', 'the', 'Peter', 'Sellars', 'production',

In [56]:
def parse_sent(sent):
    token_sent = nltk.word_tokenize(sent)
    tagged_sent = nltk.pos_tag(token_sent)

    print(tagged_sent)

    I_words = [word for word, tag in tagged_sent if tag.startswith('IN') or tag.startswith('TO')]
    PN_words = [word for word, tag in tagged_sent if tag.startswith('NNP')]
    N_words = [word for word, tag in tagged_sent if tag.startswith('NN') and not tag.startswith('NNP')]
    V_words = [word for word, tag in tagged_sent if tag.startswith('V')]
    DT_words = [word for word, tag in tagged_sent if tag.startswith('DT')]
    PRS_words = [word for word, tag in tagged_sent if tag.startswith('PR') and tag.endswith('$')] 
    PR_words = [word for word, tag in tagged_sent if tag.startswith('PR') and not tag.endswith('$')]
    J_words = [word for word, tag in tagged_sent if tag.startswith('JJ') or tag.startswith('VBG')]
    CD_words = [word for word, tag in tagged_sent if tag.startswith('CD')]
    CC_words = [word for word, tag in tagged_sent if tag.startswith('CC')]
    R_words = [word for word, tag in tagged_sent if tag.startswith('RB')]
    PS_words = [word for word, tag in tagged_sent if tag.startswith('POS')]

    def list2str(l):
        if len(l) == 0:
            return 'N/A'
        return '"' + '" | "'.join(set(l)) + '"'

    DT= list2str(DT_words)
    I = list2str(I_words)
    N = list2str(N_words)
    PN = list2str(PN_words)
    V = list2str(V_words)
    PR= list2str(PR_words)
    PRS=list2str(PRS_words)
    J = list2str(J_words)
    R = list2str(R_words)
    CD= list2str(CD_words)
    CC= list2str(CC_words)
    PS= list2str(PS_words)   

    S_pattern = "NP VP '.' | PR VP '.' | PN VP '.'" + \
                " | PP ',' NP VP '.' | PP ',' PN VP '.' | PP PR VP '.' | PP PR VP ',' CC VP '.'"
    PP_pattern = "I NP | I CD | I clause"
    NP_pattern = "N PP" + \
                 " | DT N | DT PN | DT N PP | DT J N | DT J N PP | DT J NP N PP" + \
                 " | PRS N PN | PRS N PP | PRS NP N PP | PRS N NP" + \
                 " | J N | J N PP" + \
                 " | PN PN | PN PN PN | PN PS PN | PN PP" + \
                 " | NP PP | NP N PP" + \
                 " | NP ',' RP ','"
    VP_pattern = "V NP PP | V NP | V R I PN | V I NP"
    RP_pattern = "R NP"
    clause = "PR VP | PP PR VP"

    grammar1 = nltk.CFG.fromstring(f"""
        S  -> {S_pattern}
        PP -> {PP_pattern}
        NP -> {NP_pattern}
        VP -> {VP_pattern}
        RP -> {RP_pattern}
        clause -> {clause}

        DT -> {DT}
        I -> {I}
        N -> {N}
        PN -> {PN}
        V -> {V}
        J -> {J}
        R -> {R}
        PR -> {PR}
        PRS -> {PRS}
        CD -> {CD}
        CC -> {CC}
        PS -> {PS}
    """)

    parser = nltk.ChartParser(grammar1)
    for tree in parser.parse(token_sent):
        print(tree, end='\n\n')

In [57]:
parse_sent("Her Santa Fe Opera debut in 2005 was as Nuria in the revised edition of Golijov's Ainadamar.")

[('Her', 'PRP$'), ('Santa', 'NNP'), ('Fe', 'NNP'), ('Opera', 'NNP'), ('debut', 'NN'), ('in', 'IN'), ('2005', 'CD'), ('was', 'VBD'), ('as', 'IN'), ('Nuria', 'NNP'), ('in', 'IN'), ('the', 'DT'), ('revised', 'JJ'), ('edition', 'NN'), ('of', 'IN'), ('Golijov', 'NNP'), ("'s", 'POS'), ('Ainadamar', 'NNP'), ('.', '.')]
(S
  (NP
    (PRS Her)
    (NP (PN Santa) (PN Fe) (PN Opera))
    (N debut)
    (PP (I in) (CD 2005)))
  (VP
    (V was)
    (I as)
    (NP
      (NP
        (PN Nuria)
        (PP (I in) (NP (DT the) (J revised) (N edition))))
      (PP (I of) (NP (PN Golijov) (PS 's) (PN Ainadamar)))))
  .)

(S
  (NP
    (PRS Her)
    (NP (PN Santa) (PN Fe) (PN Opera))
    (N debut)
    (PP (I in) (CD 2005)))
  (VP
    (V was)
    (I as)
    (NP
      (PN Nuria)
      (PP
        (I in)
        (NP
          (NP (DT the) (J revised) (N edition))
          (PP (I of) (NP (PN Golijov) (PS 's) (PN Ainadamar)))))))
  .)

(S
  (NP
    (PRS Her)
    (NP (PN Santa) (PN Fe) (PN Opera))
    (N debut)


In [58]:
parse_sent("She sang on the subsequent Deutsche Grammophon recording of the opera.")

[('She', 'PRP'), ('sang', 'VBD'), ('on', 'IN'), ('the', 'DT'), ('subsequent', 'JJ'), ('Deutsche', 'NNP'), ('Grammophon', 'NNP'), ('recording', 'NN'), ('of', 'IN'), ('the', 'DT'), ('opera', 'NN'), ('.', '.')]
(S
  (PR She)
  (VP
    (V sang)
    (I on)
    (NP
      (DT the)
      (J subsequent)
      (NP (PN Deutsche) (PN Grammophon))
      (N recording)
      (PP (I of) (NP (DT the) (N opera)))))
  .)



In [59]:
parse_sent("For his opera Doctor Atomic, Adams rewrote the role of Kitty Oppenheimer, originally a mezzo-soprano role, for soprano voice.")

[('For', 'IN'), ('his', 'PRP$'), ('opera', 'NN'), ('Doctor', 'NNP'), ('Atomic', 'NNP'), (',', ','), ('Adams', 'NNP'), ('rewrote', 'VBP'), ('the', 'DT'), ('role', 'NN'), ('of', 'IN'), ('Kitty', 'NNP'), ('Oppenheimer', 'NNP'), (',', ','), ('originally', 'RB'), ('a', 'DT'), ('mezzo-soprano', 'JJ'), ('role', 'NN'), (',', ','), ('for', 'IN'), ('soprano', 'JJ'), ('voice', 'NN'), ('.', '.')]
(S
  (PP (I For) (NP (PRS his) (N opera) (NP (PN Doctor) (PN Atomic))))
  ,
  (PN Adams)
  (VP
    (V rewrote)
    (NP
      (NP (DT the) (N role))
      (PP
        (I of)
        (NP
          (NP (PN Kitty) (PN Oppenheimer))
          ,
          (RP (R originally) (NP (DT a) (J mezzo-soprano) (N role)))
          ,)))
    (PP (I for) (NP (J soprano) (N voice))))
  .)

(S
  (PP (I For) (NP (PRS his) (N opera) (NP (PN Doctor) (PN Atomic))))
  ,
  (PN Adams)
  (VP
    (V rewrote)
    (NP
      (DT the)
      (N role)
      (PP
        (I of)
        (NP
          (NP (PN Kitty) (PN Oppenheimer))
        

', and Rivera sang the rewritten part of Kitty Oppenheimer at Lyric Opera of Chicago, De Nederlandse Opera, and the Metropolitan Opera., all in 2007'

In [60]:
parse_sent("Rivera sang the rewritten part of Kitty Oppenheimer at Lyric Opera of Chicago, De Nederlandse Opera, and the Metropolitan Opera., all in 2007")

[('Rivera', 'NNP'), ('sang', 'VBD'), ('the', 'DT'), ('rewritten', 'JJ'), ('part', 'NN'), ('of', 'IN'), ('Kitty', 'NNP'), ('Oppenheimer', 'NNP'), ('at', 'IN'), ('Lyric', 'NNP'), ('Opera', 'NNP'), ('of', 'IN'), ('Chicago', 'NNP'), (',', ','), ('De', 'NNP'), ('Nederlandse', 'NNP'), ('Opera', 'NNP'), (',', ','), ('and', 'CC'), ('the', 'DT'), ('Metropolitan', 'NNP'), ('Opera.', 'NNP'), (',', ','), ('all', 'DT'), ('in', 'IN'), ('2007', 'CD')]


In [48]:
parse_sent("After he became a minister he went back to Amsterdam, and took place as a sort of chairing mayor of this city.")

[('After', 'IN'), ('he', 'PRP'), ('became', 'VBD'), ('a', 'DT'), ('minister', 'NN'), ('he', 'PRP'), ('went', 'VBD'), ('back', 'RB'), ('to', 'TO'), ('Amsterdam', 'NNP'), (',', ','), ('and', 'CC'), ('took', 'VBD'), ('place', 'NN'), ('as', 'IN'), ('a', 'DT'), ('sort', 'NN'), ('of', 'IN'), ('chairing', 'VBG'), ('mayor', 'NN'), ('of', 'IN'), ('this', 'DT'), ('city', 'NN'), ('.', '.')]
(S
  (PP
    (I After)
    (clause (PR he) (VP (V became) (NP (DT a) (N minister)))))
  (PR he)
  (VP (V went) (R back) (I to) (PN Amsterdam))
  ,
  (CC and)
  (VP
    (V took)
    (NP
      (N place)
      (PP
        (I as)
        (NP
          (NP
            (NP (DT a) (N sort))
            (PP (I of) (NP (J chairing) (N mayor))))
          (PP (I of) (NP (DT this) (N city)))))))
  .)

(S
  (PP
    (I After)
    (clause (PR he) (VP (V became) (NP (DT a) (N minister)))))
  (PR he)
  (VP (V went) (R back) (I to) (PN Amsterdam))
  ,
  (CC and)
  (VP
    (V took)
    (NP
      (N place)
      (PP
        (I a

In [204]:
small_grammar = nltk.CFG.fromstring("""
    S -> NP VP
    VP -> V PR
    NP -> 'I'
    
    PR -> 'you' | 'hate'
    V -> 'hate' | 'you'
""")

In [205]:
small_parser = nltk.ChartParser(small_grammar)

In [206]:
for tree in small_parser.parse(['I', 'hate', 'you']):
    print(tree)

(S (NP I) (VP (V hate) (PR you)))
